In [7]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import login

# LangChain core
from langchain import hub
from langchain.chains import LLMChain


from huggingface_hub import login

from langchain_community.vectorstores import Chroma
from platform import python_version
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor



# Use dotenv to load your environment variables (if running in Jupyter, `%dotenv` is fine)
from dotenv import load_dotenv
load_dotenv()  # loads .env file into environment variables


True

In [4]:
# Confirm your key is loaded
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key is None:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables")
print(f"OPENROUTER_API_KEY loaded: {'Yes' if api_key else 'No'}")

api_token = os.getenv("HUGGING_FACE_TOKEN")
if api_token is None:
    raise ValueError("HUGGING_FACE_TOKEN not found in environment variables")
print(f"Hugging Face token loaded: {'Yes' if api_token else 'No'}")

OPENROUTER_API_KEY loaded: Yes
Hugging Face token loaded: Yes


In [5]:
# Initialize OpenAI client with your OPENROUTER_API_KEY
client = OpenAI(api_key=api_key)

# Initialize LangChain ChatOpenAI model with your desired parameters
llm = ChatOpenAI(model_name='meta-llama/llama-3.3-70b-instruct:free',
                  temperature=0, max_tokens=200, openai_api_key=api_key,
                  openai_api_base="https://openrouter.ai/api/v1")

In [6]:
llm.invoke("what model i use now?")

AIMessage(content="You're currently interacting with me, a language model designed to simulate conversation and answer questions to the best of my knowledge. My model is based on a transformer architecture, specifically a variant of the BERT (Bidirectional Encoder Representations from Transformers) model.\n\nHere are some details about the model I'm using:\n\n* **Model name:** LLaMA (Large Language Model Application)\n* **Model type:** Transformer-based language model\n* **Training data:** A massive dataset of text from various sources, including books, articles, and websites\n* **Parameters:** My model has approximately 1.5 billion parameters, which allows me to understand and generate human-like text\n* **Capabilities:** I can respond to a wide range of questions and topics, from simple queries to more complex conversations\n\nKeep in mind that I'm a machine learning model, and while I strive to provide accurate and helpful responses, I'm not perfect and can make mistakes. If you hav

In [8]:
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="en",
                                                                  top_k_results=1))

In [9]:
embedding = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m")

In [10]:
vectorstore = Chroma(persist_directory="J:/Vector Store/health_supplements_chromadb_store",
                                 embedding_function=embedding)

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Health Supplements Retriever", 
                                       description = '''Search for information about Health Supplements.
                                                        For any questions about Health and Gym Supplements, you must use this tool''')

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_28960\3899572792.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="J:/Vector Store/health_supplements_chromadb_store",


In [11]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [12]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [13]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

In [14]:
agent = create_tool_calling_agent(llm = llm, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [15]:
agent_executor = AgentExecutor(agent = agent, 
                               tools = tools, 
                               verbose = True, 
                               return_intermediate_steps = True)

In [17]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.16The version of Python you are currently using is 3.10.16.

> Finished chain.
The version of Python you are currently using is 3.10.16.


In [23]:
question = "Could you tell me the version of Python I'm currently using?"
response = agent_executor.invoke({'input': question})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.16The version of Python you are currently using is 3.10.16.

> Finished chain.
The version of Python you are currently using is 3.10.16.


In [24]:
response

{'input': "Could you tell me the version of Python I'm currently using?",
 'output': 'The version of Python you are currently using is 3.10.16.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'chatcmpl-tool-d874a73594484e66bea072995c73d54f', 'function': {'arguments': None, 'name': 'get_python_version'}, 'type': 'function'}]}, response_metadata={}, id='run--80fef1b6-8fae-4da7-b5a3-b15f63ae2f7b', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': 'chatcmpl-tool-d874a73594484e66bea072995c73d54f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 648, 'output_tokens': 14, 'total_tokens': 662, 'input_token_details': {}, 'output_token_details': {}}, tool_call_chunks=[{'name': 'get_python_version', 'args': None, 'id': 'chatcmpl-tool-d874a73594484e66bea072995c73d54f', 'index': 0, 'type': 'tool_ca

In [25]:
question = "Why are omega-3 fatty acids considered essential for humans?"
response = agent_executor.invoke({'input': question})



> Entering new AgentExecutor chain...

Invoking: `Health Supplements Retriever` with `{'query': 'omega-3 fatty acids essential for humans'}`


additional two studies have suggested increased mortality in smokers consuming β-carotene 
supplements (18,19). Excess vitamin A supplementation has been suggested to be associated 
with adverse effects on bone health including low bone mineral density and increased 
fracture risk (20). In addition, women consuming large amounts of vitamin A supplements 
during pregnancy have been reported to have increased incidence of congenital 
abnormalities (21). There is also a case report of intra-hepatic cholestasis in a patient with 
chronic hypervitaminosis A after 12 years of supplement consumption which resolved after 
supplements were ceased (22). In addition to toxicity from excess vitamin consumption,

reproductive toxicity, infertility, demasculinization and increased promotion of estrogen-
responsive cancers such as breast and endometrial canc

In [26]:
print(response['output'])

Agent stopped due to max iterations.


In [ ]:
question = "What is the longest river in the world in the world?"
response = agent_executor.invoke({'input': question})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'longest river in the world'}`


Page: List of river systems by length
Summary: This is a list of the longest rivers on Earth. It includes river systems over 1,000 kilometres (620 mi) in length.


Invoking: `wikipedia` with `{'query': 'Amazon River'}`


Page: Amazon River
Summary: The Amazon River (US: ; Spanish: Río Amazonas, Portuguese: Rio Amazonas) in South America is the largest river by discharge volume of water in the world, and the longest or second-longest river system in the world, a title which is disputed with the Nile.
The headwaters of the Apurímac River on Nevado Mismi had been considered, for nearly a century, the Amazon basin's most distant source until a 2014 study found it to be the headwaters of the Mantaro River on the Cordillera Rumi Cruz in Peru. The Mantaro and Apurímac rivers join, and with other tributaries form the Ucayali River, which in turn meets the Marañón River upstream of Iq

In [ ]:
print(response['output'])

The title of the longest river in the world is disputed between the Amazon River and the Nile River. 

- The **Amazon River**, located in South America, has a recognized length of approximately **6,400 km (4,000 miles)**, but some reports suggest it could be as long as **6,575–7,062 km (4,086–4,388 miles)**. It is also the largest river by discharge volume of water in the world.

- The **Nile River**, primarily flowing through northeastern Africa, is traditionally considered the longest river, with a length of about **6,650 km (4,130 miles)**.

Due to the varying measurements and definitions of river lengths, both rivers are often cited as contenders for the title of the longest river in the world.


OpenAI Seceond Model

In [ ]:
question = "What are the side effects of taking too much vitamin D?"
response = agent_executor.invoke({'input': question})
response



> Entering new AgentExecutor chain...

Invoking: `HHealth_Supplements` with `{'query': 'side effects of taking too much vitamin D'}`


beneficial, recent reviews of vitamin and mineral supplement trials in community-dwelling 
adults with no nutritional deficiencies have concluded that there is no clear evidence of 
beneficial health effects. These include primary or secondary prevention of chronic diseases 
including cardiovascular disease, cancer, cognitive decline and effects on overall mortality 
(10,11). Indeed, on the contrary, there is evidence for possible harm based on consumption 
of individual vitamins and mineral in excess. Toxicity following consumption of water-
soluble vitamins is rare. However, photosensitivity and neurotoxicity have been reported at 
doses higher than 500 mg/d of pyridoxine (vitamin B6) (12) and cases of pyridoxine-

soluble vitamins is rare. However, photosensitivity and neurotoxicity have been reported at 
doses higher than 500 mg/d of pyridoxine (v

{'input': 'What are the side effects of taking too much vitamin D?',
 'output': "Taking too much vitamin D can lead to several side effects, primarily due to vitamin D toxicity. Here are some potential side effects of excessive vitamin D intake:\n\n1. **Hypercalcemia**: Elevated levels of calcium in the blood, which can cause nausea, vomiting, weakness, and frequent urination.\n\n2. **Kidney Damage**: Excessive vitamin D can lead to kidney stones and calcification of the kidneys, potentially resulting in kidney damage.\n\n3. **Bone Pain**: Paradoxically, too much vitamin D can lead to bone pain and discomfort.\n\n4. **Gastrointestinal Issues**: Symptoms may include nausea, vomiting, constipation, and loss of appetite.\n\n5. **Fatigue and Weakness**: High levels of vitamin D can cause general fatigue and muscle weakness.\n\n6. **Confusion and Disorientation**: Severe cases of vitamin D toxicity can lead to confusion or disorientation.\n\nIt's important to consult with a healthcare provi

In [ ]:
print(response['output'])

Taking too much vitamin D can lead to several side effects, primarily due to vitamin D toxicity. Here are some potential side effects of excessive vitamin D intake:

1. **Hypercalcemia**: Elevated levels of calcium in the blood, which can cause nausea, vomiting, weakness, and frequent urination.

2. **Kidney Damage**: Excessive vitamin D can lead to kidney stones and calcification of the kidneys, potentially resulting in kidney damage.

3. **Bone Pain**: Paradoxically, too much vitamin D can lead to bone pain and discomfort.

4. **Gastrointestinal Issues**: Symptoms may include nausea, vomiting, constipation, and loss of appetite.

5. **Fatigue and Weakness**: High levels of vitamin D can cause general fatigue and muscle weakness.

6. **Confusion and Disorientation**: Severe cases of vitamin D toxicity can lead to confusion or disorientation.

It's important to consult with a healthcare provider before taking vitamin D supplements, especially
